### Generating data for statistic testing

Over the next few weeks we will continue to learn to apply new statistical tests using tools like machine-learning, maximum likelihood, linear regression, and Bayesian statistics. In most of these, we will have a clear  *generative model* defined that we are trying to fit. In others, the generative model may be more abstract, such as when analyzing a set of images, but we still know that the images can be represented as a certain type of data that can be generated. It is important to think deeply about the structure of your data and how your analytical tools are extracting information from it. The easiest way to do this, both as a learning tool, and to validate for yourself and others that your methods are working as intended, is to generate simulated data to run on your analyses as a test case. 

In [1]:
import numpy as np
import pandas as pd
import toyplot


### An example
Let's say that we are trying to fit a linear model that has an intercept and a slope. We can use the numpy random library to easily generate a large set of data that is genarated from a defined intercept and slope which we can then use in our model. We expect the model should return the pre-defined intercept and slope that we used to create the data. If they match, then we have validated that the method is working how we expect it to. We may want to test several generated data sets to be sure. 

Let's use an example linear regression where the intercept is a variable alpha ($\alpha$), the slope is a variable beta ($\beta$), there is a component of noise in the data ($\sigma$). 

$ y = \beta X + \alpha + \sigma $

In [2]:
# the true values we will use
true_beta = 4.5
true_alpha = -1.5
noise = 2.0

# generate a bunch of observations
nsize = 1000
xgen = np.random.normal(0, 2, nsize)
sgen = np.random.normal(0, noise, nsize)
ygen = true_beta * xgen + true_alpha + sgen

#### 1. Examine the data table

In [3]:
# dataframes may be used just to view the data nicely
data = pd.DataFrame({
    "X": xgen,
    "y": ygen,
})
data.head()

,X,y
0,1.461812,7.803337
1,0.750074,1.140496
2,-2.056346,-8.260684
3,3.642154,16.334200
4,-0.427907,-2.418581


#### 2. Examine the data using plotting

In [4]:
# plot the data
toyplot.scatterplot(xgen, ygen, height=250, width=300, size=3);

<g class="toyplot-Datum" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:1.0;stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0" transform="translate(125.27836518254293, 151.737623621

### Applying machine learning 

The general workflow for scikit-learn involves 4 things: 
1.  prepare the data
2. initialize a Model class instance
3. fit training data with the model
4. predict test data with the model

#### 1. Preparing the data
The features must be a (nsamples, nfeatures) array, and the labels should be a (nsamples, 1) array. You should also split the data set into a training set and a test set. The training data is what the model will be optimized to make predictions for and the test set is used to validate how well it performs. 

In [5]:
# how many samples to hold back for testing
tsize = 200

In [7]:
data.X.values.shape
# needs to be 2 dimensional array 


(1000,)

In [8]:
# needs to be 2 dimensional array 
data.X.values[:, None].shape

(1000, 1)

In [10]:
# convert to a 2d array
X = data.X.values[:, None]# .ravel not needed - this reduces dimensions

# separate test from training
X_test = X[:tsize]
X_train = X[tsize:]

# show
print(X.shape)
print(X[:5])

(1000, 1)
[[ 1.46181176]
 [ 0.75007387]
 [-2.05634588]
 [ 3.64215442]
 [-0.42790706]]


In [11]:
# convert to a 1d array
y = data.y.values

# separate test from training
y_test = y[:tsize]
y_train = y[tsize:]

# show
print(y.shape)
print(y[:5])

(1000,)
[  7.80333674   1.14049597  -8.26068431  16.33419995  -2.41858099]


#### 2. initialize a model instance

In [12]:
from sklearn.linear_model import LinearRegression # choose model of linear regression 
model = LinearRegression(fit_intercept=True) # choose hyperparameters

#### 3. fit the model

In [13]:
model.fit(X_train, y_train) # these are the labels for the supervised method
# already knows it is using a linear regression relationship from above
# trains in place doesnt return a copy 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

#### 4. predict y for new data X

In [15]:
# get predicted 'y' values for held back data X_test
yfit = model.predict(X_test)
yfit # these are predictions of y based on the values of x we gave it 

array([  5.04659453,   1.82241457, -10.89069582,  14.92356928,
        -3.51385118, -22.59952975,  10.33792409, -18.05375024,
        -5.36139203,  26.14730658, -17.89942955,  -2.24395259,
        -9.18968361,  -8.99127681,  -1.95065172,  -2.60482975,
       -10.91525137,  -0.66214681,  -1.05159568,  -7.60603665,
        -0.50078844,   6.4783115 ,   5.24865783,  -5.05365976,
        -9.03544968,  -6.01649139, -11.91411081, -13.72183027,
        -4.64926575,  -3.74538464,   5.89876055, -18.67691966,
        -2.6726217 ,   4.01070835,   8.33659499,  -2.23597896,
        -2.54816344,  -8.34777401,  -1.01885651,   2.54120034,
         1.04380065,  15.81764487, -14.59709068, -12.6094901 ,
        -4.06151628,   3.00492329,  -6.71715768,  11.59625031,
         6.19324483,  -6.5297504 ,   9.6688716 ,  -3.78951048,
        -3.12412758,   2.20257405,   0.11396678, -13.01478443,
        -0.30129187,  -2.85014536,   1.01749696,   4.2133155 ,
         0.77845932, -15.25498608,  -4.35868833,   1.72

### Assess the goodness of fit (score)
This is the typical way to assess how well a machine learning algorithm has performed, and is called validation. We are asking how close the predicted values of y are to the known values of y, which we know because we held back these data from the analyses (the test data set). Below is the r2 value and mean squared error. 

In [16]:
# compute r2 from comparing predicted y to actual y
from sklearn.metrics import r2_score, mean_squared_error

results ={
    "R2": r2_score(yfit, y_test),
    "MSE": mean_squared_error(yfit, y_test),
}
print(results)
# high r2 given larger errors given 

{'R2': 0.95377627654714603, 'MSE': 3.526821785071959}


### Assess that we've run the model correctly

Here, because we simulated the data, we can perform another type of validation, which is to ask whether the parameters inferred by the model match to those that we used in our generative model to simulate the data. If so, then we can have improved confidence that we have chosen an appropriate model. The practice of creating simulated data to apply to a model is a useful exercise in trying to understand how the model analyzes the features of the data. Here we generated data with a clear linear relationship, and so clearly a LinearRegression model was appropriate. But other models could predict the y values accurately as well, using different approaches that similarly aim to minimize the error on the training data set (X_train) when some function is applied to it compared to the training labels (y_train). In LinearRegression the function to minimize is the squared error, which is the same process employed in any ordinary least squares model. In other models we will see the process can be different, but lead to similar results. 

In [17]:
pd.DataFrame({
    "Beta": [true_beta, model.coef_[0]],
    "alpha": [true_alpha, model.intercept_],
    }, 
    index=["true", "estimated"])
# estimated parameters of model will end with underscore _

,Beta,alpha
true,4.50000,-1.500000
estimated,4.53001,-1.575428


### Plot model fit

In [18]:
# build canvas
c = toyplot.Canvas(height=300, width=350)
a = c.cartesian()

# add training and test data points
a.scatterplot(X_train[:, 0], y_train, size=4, opacity=0.5);
a.scatterplot(X_test[:, 0], y_test, size=4, opacity=0.5);

# show that fitted line
a.plot(X_test[:, 0], yfit, color='black', style={"stroke-width": 2.5});

<g clas